# Import Libraries

In [10]:
!pip install keras==2.3.1

In [11]:
import os
import cv2
import seaborn as sns
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import save_model

# Import Dataset

In [12]:
directory = '../input/homerbart1/homer_bart_1'  #path of extracted dataset (current location)
files = [os.path.join(directory, f) for f in sorted(os.listdir(directory))] #contains paths of each images

# Data Preprocessing

In [13]:
height, width = 128, 128   #since all images are of different shapes reshape them to this shape
images =[]  #pixel values of all images
classes = []    #class of all images

for image_path in files:
    #print(image_path)

    try:    #exception handling for the irrelevant file DSstore that is in the directory (shortly can delete the file)
        image = cv2.imread(image_path)  #read image one by one
        (H, W) = image.shape[:2]    #since this is RGB image need only first two values are height and weight
    except:
        continue
    
    image = cv2.resize(image, (width, height))  #resize the image to 128,128,3
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) #convert to gray scale
    #cv2.imshow('', image)
    #cv2.waitKey(0)
    image = image.ravel()       #convert matrix to vector to feed to input layer
    images.append(image)    #store all vector representation of images
    
    #print(os.path.normpath(image_path)) #normpath removes redundant separator to extract filename
    #print(os.path.basename(os.path.normpath(image_path)))   #basename - only name (without location)
    
    image_name = os.path.basename(os.path.normpath(image_path)) #extract only filename
    
    if(image_name.startswith('b')): #encoding categorical data 
        class_name = 0  #bart - 0
    else:
        class_name = 1  #homer -1

    classes.append(class_name)  #store all class_names
    #print(class_name)

#convert images and classes to np representation
X = np.asarray(images)  #or can use X = np.array(images)
Y = np.asarray(classes) #or can use Y = np.array(classes)

# Visualise Counts

In [14]:
sns.countplot(x=Y)  #dis;play barchart
print(np.unique(Y, return_counts=True)) #to see number of images in each class

# Normalising The Data

In [15]:
scaler = MinMaxScaler()
X = scaler.fit_transform(X)                 #or just divide X by 255

# Train Test Split

In [16]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state = 1)    #20% data as test data

# Train Model

In [17]:
#number of neurones in hidden layer = (inp layer + out layer )/ 2
#(16384 + 2) /2

#building the model
network1 = tf.keras.models.Sequential()
network1.add(tf.keras.layers.Dense(input_shape=(X_train.shape), units = 8193, activation='relu'))
network1.add(tf.keras.layers.Dense(units = 8193, activation='relu'))    #2nd layer
network1.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))  #since binary classification sigmoid activation

#visualise the model
network1.summary()

#compile model
network1.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

#train
history = network1.fit(X_train, Y_train, epochs = 50)

# Model Evaluation

In [18]:
print(history.history.keys())
plt.plot(history.history['loss']);

# Testing

In [19]:
predictions = network1.predict(X_test)

#convert predictions (probabilities) to classes
predictions = (predictions > 0.5) #threshold = 0.5

#test evaluation
print('Accuracy :', accuracy_score(predictions, Y_test)*100)
cm = confusion_matrix(Y_test, predictions)

In [20]:
#display heatmap
sns.heatmap(cm, annot=True);

In [21]:
#classification report
print(classification_report(Y_test, predictions))

# Save Model & Weights

In [22]:
#save json model structure
model_json = network1.to_json()   #convert model to json format
with open('network1.json', 'w') as json_file:
  json_file.write(model_json) #write the json format of model to network1.json file

#save model weights
network1_saved = tf.keras.models.save_model(network1, 'weights1.hdf5')

# Load Model & Weights

In [23]:
#read stored json model structure
with open('network1.json', 'r') as json_file:
  json_saved_model = json_file.read()

#build model from json structure
network1_loaded = tf.keras.models.model_from_json(json_saved_model)   #convert json format of stored model to model
network1_loaded.load_weights('weights1.hdf5') #load weights stored before
network1_loaded.compile(optimizer='Adam', loss='binary_crossentropy', metrics =['accuracy']) #setting exact optimizer, loss, metrics

#model summary
network1_loaded.summary()

# Single Image Prediction

In [24]:
test_image = X_test[0]
test_image = scaler.inverse_transform(test_image.reshape(1, -1))
if(network1_loaded.predict(test_image)[0][0] < 0.5):
  print('Bart')
else:
  print('Homer')